In [85]:
import pandas as pd
import pyodbc
from datetime import datetime

In [86]:
sql_query = """
    -- all renewal WO and their flocid
    SELECT
        [WorkOrderNumber],
        [OrderType],
        [CompanyCode],
        [FunctionLocation],
        CASE
            WHEN [TechCompletionDate] IS NULL THEN [BasicFinishDate]
            ELSE [TechCompletionDate]
        END AS 'CompletionDate',
        [MaintenanceActivityType],
        [MaintenanceActivityTypeDesc],
        [MainUserStatus],
        [MainUserStatusDesc],
        [ActualTotalCost]
    FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_WorkOrder]
    WHERE
        OrderType = 'MW04'
        AND CompanyCode = '5000'
        AND MainUserStatusDesc = 'Practically Completed'
        AND ActualTotalCost IS NOT NULL
        AND ActualTotalCost > 0
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';DATABASE=myANALYTICS_SP;Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_renewal_wo = pd.read_sql_query(sql_query, conn)

# all_renewal_wo.to_pickle("../pkl/work_order_all.pkl")
# Display the DataFrame
all_renewal_wo['CompletionDate'] = pd.to_datetime(all_renewal_wo['CompletionDate'])

all_renewal_wo = all_renewal_wo[['FunctionLocation', 'ActualTotalCost']]
all_renewal_wo

C:\Users\R893859\AppData\Local\Temp\ipykernel_17612\2637160193.py:33: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_renewal_wo = pd.read_sql_query(sql_query, conn)


,FunctionLocation,ActualTotalCost
0,NL-02ML,168659.53
1,NL-02ML,266823.32
2,LU000037,587.81
3,LU000040,2291.71
4,LU000034,2036.69
...,...,...
10060,MA-03ML,1137917.24
10061,TO000303,57125.67
10062,GA-02ML,2090.00
10063,GA-01ML,1045.00


In [107]:
# all_structure_floc
sql_query = """
	-- Converting Civil Structures Asset Renewal from Python to SQL
	WITH DF1 AS(
		SELECT [FuncLocID]
			,[STRUCTURE_LENGTH]
			,[YEAR_OF_CONSTRUCTION]
			,[LinearRefPattern]
			,(CASE
					WHEN [ASSET_CLASS_TYPE] = 'BRIDGE' AND [SPAN_MATERIAL] = 'STEEL' THEN 'Steel Bridge'
					WHEN [ASSET_CLASS_TYPE] = 'BRIDGE' AND [SPAN_MATERIAL] = 'TIMBER' THEN 'Timber Bridge'
					ELSE NULL
				END) AS [Asset_Type_Bridge]
			,(CASE
					WHEN [STRUCTURE_TYPE] = 'REINFORCED CONCRETE BOXCULVERT' AND CAST(SUBSTRING(CAST([YEAR_OF_CONSTRUCTION] AS VARCHAR), 1, 4) AS FLOAT) < 2000 THEN 'Reinforced Concrete Box Culvert (RCBC) prior to year 2000'
					WHEN [STRUCTURE_TYPE] = 'REINFORCED CONCRETE BOXCULVERT' AND CAST(SUBSTRING(CAST([YEAR_OF_CONSTRUCTION] AS VARCHAR), 1, 4) AS FLOAT) >= 2000 THEN 'Reinforced Concrete Box Culvert (RCBC) designed to AS1597'
					ELSE NULL
				END) AS Asset_Type_Boxculvert
			,(CASE 
					WHEN [STRUCTURE_TYPE] = 'REINFORCED PIPE CULVERT' THEN 'Reinforced Concrete Pipe (RCP)'
					WHEN [STRUCTURE_TYPE] = 'CORRUGATED METAL PIPE' THEN 'Corrugated Metal Pipe (CMP)'
					WHEN [STRUCTURE_TYPE] != 'REINFORCED PIPE CULVERT' AND [SPAN_TYPE] = 'PIPE CULVERT' AND SPAN_MATERIAL LIKE '%CONCRETE%' THEN 'Concrete Pipe (Unreinforced)'
					ELSE NULL
				END) AS [Asset_Type_Pipe]
			,(CASE 
					WHEN [SPAN_MATERIAL] = 'CAST-IN-SITU CONCRETE' THEN 'Concrete In situ Arch'
					ELSE NULL
				END) AS [Asset_Type_Arch]
			,(CASE 
					WHEN [FuncLocID] LIKE 'RW%' THEN 'Concrete Retaining Wall'
					ELSE NULL
				END) AS [Asset_Type_Wall]
			,(CASE 
					WHEN SPAN_MATERIAL = 'STEEL' AND STRUCTURE_USAGE = 'PEDESTRIAN' AND TechnicalObjectTypeDesc = 'Bridges' THEN 'Pedestrian Overbridge (Steel)'
					ELSE NULL
				END) AS [Asset_Type_Overbridge]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.civil.structure.ringfenced].[vw_FLOCStructure]
		WHERE 1=1
		AND [UserStatusDesc]  = 'In Service'
	),

	DF2 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[STRUCTURE_LENGTH]
			,[YEAR_OF_CONSTRUCTION]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[STRUCTURE_LENGTH]
				,[YEAR_OF_CONSTRUCTION]
				,[LinearRefPattern]
				,CAST([Asset_Type_Bridge] AS VARCHAR(100)) AS [Asset_Type_Bridge]
				,CAST([Asset_Type_Boxculvert] AS VARCHAR(100)) AS [Asset_Type_Boxculvert]
				,CAST([Asset_Type_Pipe] AS VARCHAR(100)) AS [Asset_Type_Pipe]
				,CAST([Asset_Type_Arch] AS VARCHAR(100)) AS [Asset_Type_Arch]
				,CAST([Asset_Type_Wall] AS VARCHAR(100)) AS [Asset_Type_Wall]
				,CAST([Asset_Type_Overbridge] AS VARCHAR(100)) AS [Asset_Type_Overbridge]
			FROM DF1) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([Asset_Type_Bridge]
				,[Asset_Type_Boxculvert]
				,[Asset_Type_Pipe]
				,[Asset_Type_Arch]
				,[Asset_Type_Wall]
				,[Asset_Type_Overbridge])
			) AS unpvt
	)

	SELECT * FROM DF2
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_structure_floc = pd.read_sql_query(sql_query, conn)

all_structure_floc['ASSET_GROUP'] = 'civil_structure'

# all_structure_floc
all_structure_floc[all_structure_floc['FuncLocID'] == 'CU400577']

C:\Users\R893859\AppData\Local\Temp\ipykernel_17612\3480579043.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_structure_floc = pd.read_sql_query(sql_query, conn)


,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern,ASSET_GROUP
2987,CU400577,Reinforced Concrete Box Culvert (RCBC) designe...,Asset_Type_Boxculvert,18,20130101,MA-16ML,civil_structure
2988,CU400577,Concrete Pipe (Unreinforced),Asset_Type_Pipe,18,20130101,MA-16ML,civil_structure
2989,CU400577,Concrete In situ Arch,Asset_Type_Arch,18,20130101,MA-16ML,civil_structure


: 

In [88]:
# all_civil_track_floc
sql_query = """
    -- Civil Track Asset Renewal Python Replication in SQL.
    WITH DF1 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN LEFT_RAIL_HARDNESS LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Sml Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + LEFT_RAIL_HARDNESS + ' Curves' + ' (200 to 600 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS NOT LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Sml Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Curves' + ' (200 to 600 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Med Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + LEFT_RAIL_HARDNESS + ' Curves' + ' (601 to 1000 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS NOT LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Med Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Curves' + ' (601 to 1000 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Lrg Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + LEFT_RAIL_HARDNESS + ' Curves' + ' (1001 to 2500 metre radius)'
                    WHEN LEFT_RAIL_HARDNESS NOT LIKE '%HH%' AND Class = 'CURVE' AND TechnicalObjectTypeDesc = 'Curves - Lrg Radius' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Curves' + ' (1001 to 2500 metre radius)'
                    WHEN Class = 'ZPM_TANGENTS' THEN LEFT_RAIL_MASS + ' Rail - ' + 'Straights'
                    ELSE NULL
                END) AS [ASSET_TYPE_RAIL]
                ,(CASE
                    WHEN SLEEPER_SIZE_VALUE = 'CON 28T' THEN '28.0 Tonne Axle Load PSC Sleepers'
                    WHEN SLEEPER_SIZE_VALUE = 'CON 22T' THEN '22.5 Tonne Axle Load PSC Sleepers'
                    WHEN SLEEPER_SIZE_VALUE = 'INTERSPERSED CONCRETE/TIMBER' OR SLEEPER_SIZE_VALUE = 'TIMBER' THEN 'Timber Sleepers'
                    WHEN SLEEPER_SIZE_VALUE = 'STEEL' THEN 'Steel Sleepers'
                    ELSE NULL
                END) AS [ASSET_TYPE_SLEEPERS]
                ,(CASE
                    WHEN SLEEPER_FASTENINGS LIKE '%PANDROL%' THEN 'Replace Pandrol E-Clip Fastenings'
                    WHEN SLEEPER_FASTENINGS = 'FIST-BTR' THEN 'Replace Fist Fastenings'
                    ELSE NULL
                END) AS [ASSET_TYPE_FASTENINGS]

        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.ringfenced].[vw_FLOCTrackFeature]
        WHERE UserStatusDesc = 'In Service'
    ),

    DF2 AS (
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_RAIL] AS VARCHAR(100)) AS [ASSET_TYPE_RAIL]
                ,CAST([ASSET_TYPE_SLEEPERS] AS VARCHAR(100)) AS [ASSET_TYPE_SLEEPERS]
                ,CAST([ASSET_TYPE_FASTENINGS] AS VARCHAR(100)) AS [ASSET_TYPE_FASTENINGS]
            FROM DF1) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_RAIL]
                ,[ASSET_TYPE_SLEEPERS]
                ,[ASSET_TYPE_FASTENINGS])
            ) AS unpvt
    ),

    DF3 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN VEE_TYPE = 'SWINGNOSE' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'Swing Nose Points and Crossing'
                    WHEN VEE_TYPE = 'RBM' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'RBM Points and Crossing'
                    WHEN VEE_TYPE = 'FABRICATED' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'Fabricated Points and Crossing'
                    WHEN VEE_TYPE = 'SPRINGWING' THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + 'Spring Wing Crossing'
                    WHEN VEE_TYPE IS NOT NULL THEN ANGLE + ' (' + LEFT_RAIL_MASS + ') ' + VEE_TYPE
                    ELSE NULL
                END) AS [ASSET_TYPE_POINTS_AND_CROSSING]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.rail.ringfenced].[vw_FLOCTurnoutExtended]
        WHERE UserStatusDesc = 'In Service'
    ),

    DF4 AS (
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_POINTS_AND_CROSSING] AS VARCHAR(100)) AS [ASSET_TYPE_POINTS_AND_CROSSING]
            FROM DF3) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_POINTS_AND_CROSSING])
            ) AS unpvt
    ),

    DF5 AS (

        SELECT [FuncLocID]
            ,'Insulated Rail Joints' AS [ASSET]
            ,'ASSET_TYPE_IRJS' AS [ASSET_TYPE]
            ,NULL AS FuncLocLinearLengthKM
            ,[LinearRefPattern]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.rail.ringfenced].[vw_FLOCIRJ]
        WHERE UserStatusDesc = 'In Service'
    ),

    RESULT AS (
        SELECT * FROM DF2
        UNION ALL
        SELECT * FROM DF4
        UNION ALL
        SELECT * FROM DF5
    )

    SELECT * FROM RESULT;
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_civil_track_floc = pd.read_sql_query(sql_query, conn)

all_civil_track_floc['ASSET_GROUP'] = 'civil_track'

# # Display the DataFrame
# all_civil_track_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_17612\164675252.py:120: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_civil_track_floc = pd.read_sql_query(sql_query, conn)


In [89]:
# all_civil_right_of_way_floc
sql_query = """
    -- Civil Right of Way Asset Renewal Python Conversion to SQL.
    WITH DF1 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN TechnicalObjectTypeDesc = 'Public LX' THEN 'Public Level Xing Refurbishment (signage, drainage, road surface)'
                    WHEN TechnicalObjectTypeDesc = 'Maintenance LX' THEN 'Occupational & Maintenance Level Xing Refurbishment (signage, drainage, road surface)'
                    WHEN FuncLocName4 LIKE '%cattle%' THEN 'Cattle Xing Refurbishment (signage, drainage, road surfacing)'
                    ELSE NULL
                END) AS [ASSET_TYPE_CROSSING]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.operationalsystem.ringfenced].[vw_FLOCLevelCrossing]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF2 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_CROSSING] AS VARCHAR(100)) AS [ASSET_TYPE_CROSSING]
            FROM DF1) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_CROSSING])
            ) AS unpvt
    ),

    DF3 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE
                    WHEN AZJ_SG_EQUIPGRP LIKE '%pedestrian%' THEN 'Pedestrian Foot Xing Refurbishment (signage, drainage, road surface)'
                    ELSE NULL
                END) AS [ASSET_TYPE_CROSSING]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.electrical.ringfenced].[vw_FLOCElectrical]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF4 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_CROSSING] AS VARCHAR(100)) AS [ASSET_TYPE_CROSSING]
            FROM DF3) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_CROSSING])
            ) AS unpvt
    ),

    RESULT AS (
        SELECT * FROM DF2
        UNION ALL
        SELECT * FROM DF4
    )

    SELECT * FROM RESULT;
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_civil_right_of_way_floc = pd.read_sql_query(sql_query, conn)

all_civil_right_of_way_floc['ASSET_GROUP'] = 'civil_rightofway'

# # Display the DataFrame
# all_civil_right_of_way_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_17612\3517091073.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_civil_right_of_way_floc = pd.read_sql_query(sql_query, conn)


In [90]:
# all_control_system_floc
sql_query = """
	-- Control Systems Asset Renewal Python Conversion to SQL.
	WITH DF1 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN [FuncLocName] LIKE '%signal gantry%' THEN 'Signal Gantry'
					ELSE NULL
				END) AS [ASSET_TYPE_GANTRY]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.civil.structure.ringfenced].[vw_FLOCStructure]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF2 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_GANTRY] AS VARCHAR(100)) AS [ASSET_TYPE_GANTRY]
			FROM DF1) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_GANTRY])
			) AS unpvt
	),

	DF3 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'TRAIN PROTECTION' AND AZJ_SG_EQUIPTYPE LIKE '%ATP%' THEN 'Train Protection Systems (ATP)'
					ELSE NULL
				END) AS [ASSET_TYPE_TRAIN_PROTECTION]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Weather Station Syst' THEN 'Weather Station'
					ELSE NULL
				END) AS [ASSET_TYPE_WEATHER_STATION]
			,(CASE
					WHEN [AZJ_SG_SYSTYPE] = 'CROSSINGS' AND [AZJ_SG_EQUIPTYPE] LIKE 'RMS%' THEN 'Level Crossing Monitors'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_MONITORS]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.track.ringfenced].[vw_FLOCTrainDetection]
		WHERE [UserStatusDesc]  = 'In Service'
	),
	
	DF4 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_TRAIN_PROTECTION] AS VARCHAR(100)) AS [ASSET_TYPE_TRAIN_PROTECTION]
				,CAST([ASSET_TYPE_WEATHER_STATION] AS VARCHAR(100)) AS [ASSET_TYPE_WEATHER_STATION]
				,CAST([ASSET_TYPE_LX_MONITORS] AS VARCHAR(100)) AS [ASSET_TYPE_LX_MONITORS]
			FROM DF3) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_TRAIN_PROTECTION],
				[ASSET_TYPE_WEATHER_STATION],
				[ASSET_TYPE_LX_MONITORS])
			) AS unpvt
	),

	DF5 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN QR_CONFIG = 'LED SIGNAL' THEN 'LED Signal Modules'
					ELSE NULL
				END) AS [ASSET_TYPE_SIGNAL_MODULES]
			,(CASE
					WHEN FuncLocName LIKE '%s2%' AND FuncLocName LIKE '%telemetry%' THEN 'S2 Telemetry Unit'
					ELSE NULL
				END) AS [ASSET_TYPE_TELEMETRY_UNIT]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'WHEEL IMPACT LOAD DETECTOR' THEN 'Wheel Impact Load Detector (WILD)'
					WHEN TechnicalObjectTypeDesc = 'Hot Bearing / Wheel' AND AZJ_SG_EQUIPTYPE LIKE '%detector%' THEN 'Hot Bearing and Hot Wheel Detectors'
					ELSE NULL
				END) AS [ASSET_TYPE_DETECTOR]
			,(CASE
					WHEN [AZJ_SG_EQUIPTYPE] = 'POLE' THEN 'Radio Poles'
					ELSE NULL
				END) AS [ASSET_TYPE_POLE]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'ELECTRIC POINTS' OR AZJ_SG_EQUIPGRP = 'HYDRAULIC POINTS' OR AZJ_SG_EQUIPGRP = 'MECHANICAL POINTS' THEN 'Points Machines'
					ELSE NULL
				END) AS [ASSET_TYPE_POINTS]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Alternator Control E' AND WorkCenterDesc LIKE '%Control Systems%' AND AZJ_SG_EQUIPGRP = 'STANDBY SUPPLY' THEN 'Diesel Standby Alternators'
					ELSE NULL
				END) AS [ASSET_TYPE_ALTERNATORS]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_FLOCTelemetry]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF6 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_SIGNAL_MODULES] AS VARCHAR(100)) AS [ASSET_TYPE_SIGNAL_MODULES]
				,CAST([ASSET_TYPE_TELEMETRY_UNIT] AS VARCHAR(100)) AS [ASSET_TYPE_TELEMETRY_UNIT]
				,CAST([ASSET_TYPE_DETECTOR] AS VARCHAR(100)) AS [ASSET_TYPE_DETECTOR]
				,CAST([ASSET_TYPE_POLE] AS VARCHAR(100)) AS [ASSET_TYPE_POLE]
				,CAST([ASSET_TYPE_POLE] AS VARCHAR(100)) AS [ASSET_TYPE_POINTS]
				,CAST([ASSET_TYPE_POLE] AS VARCHAR(100)) AS [ASSET_TYPE_ALTERNATORS]
			FROM DF5) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_SIGNAL_MODULES],
				[ASSET_TYPE_TELEMETRY_UNIT],
				[ASSET_TYPE_DETECTOR],
				[ASSET_TYPE_POLE],
				[ASSET_TYPE_POINTS],
				[ASSET_TYPE_ALTERNATORS])
			) AS unpvt
	),

	DF7 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'UPS' THEN 'UPS'
					ELSE NULL
				END) AS [ASSET_TYPE_UPS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'RELAY INTERLOCKING' THEN 'Relay Interlockings'
					ELSE NULL
				END) AS [ASSET_TYPE_INTERLOCKINGS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'DC TRACK CIRCUIT' THEN 'DC and Impulse Track Circuits'
					WHEN AZJ_SG_EQUIPTYPE = 'JS TRACK CIRCUIT' THEN 'Jointless Track Circuits'
					ELSE NULL
				END) AS [ASSET_TYPE_TRACK_CIRCUITS]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Axle Counter' THEN 'Axle Counters'
					ELSE NULL
				END) AS [ASSET_TYPE_AXLE_COUNTERS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'DRAGGING EQUIPMENT DETECTOR' THEN 'Dragging Equipment Detector'
					ELSE NULL
				END) AS [ASSET_TYPE_DRAGGING_EQUIPMENT]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'ELECTRONIC WEIGHBRIDGE' THEN 'Weighbridges'
					ELSE NULL
				END) AS [ASSET_TYPE_WEIGHBRIDGES]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'Level Crossing' AND AZJ_SG_EQUIPTYPE LIKE '%camera%' THEN 'Level Crossings Cameras'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_CAMERAS]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'LX PROTECTION SYSTEM' THEN 'Over Height Level Crossing Protection'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_PROTECTION]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'CQCN DATA NETWORK' AND (AZJ_SG_EQUIPGRP LIKE '%switch%' OR AZJ_SG_EQUIPGRP LIKE '%router%') THEN 'Data - LAN (Switch & Routers)'
					ELSE NULL
				END) AS [ASSET_TYPE_DATA_LAN]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'RADIO SYSTEMS' AND AZJ_SG_SYSTYPE = 'TETRA' THEN 'Radio Systems - TETRA'
					ELSE NULL
				END) AS [ASSET_TYPE_RADIO_SYSTEMS]
			,(CASE
					WHEN AZJ_SG_SYSGRP = 'RADIO SYSTEMS' AND AZJ_SG_SYSID LIKE '%ded%' THEN 'Radio DEDs'
					ELSE NULL
				END) AS [ASSET_TYPE_RADIO_DEDS]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP LIKE '%antenna%' AND AZJ_SG_EQUIPTYPE LIKE '%pole%' THEN 'Radio Poles'
					ELSE NULL
				END) AS [ASSET_TYPE_RADIO_POLES]
			,(CASE
					WHEN TechnicalObjectTypeDesc = 'DMR' THEN 'Digital Microwave Radio (DMR)'
					ELSE NULL
				END) AS [ASSET_TYPE_DMR]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'SDH' OR AZJ_SG_EQUIPGRP = 'PDH' THEN 'Active Terminal Equipment (PDH, SDH)'
					ELSE NULL
				END) AS [ASSET_TYPE_TERMINAL_EQUIPMENT]
			,(CASE
					WHEN AZJ_SG_EQUIPTYPE = 'COMMS EQUIPMENT ROOM' THEN 'Communication Equipment Rooms'
					ELSE NULL
				END) AS [ASSET_TYPE_EQUIPMENT_ROOMS]
			,(CASE
					WHEN AZJ_SG_EQUIPGRP = 'HOUSING' AND WorkCenterDesc LIKE '%control systems%' THEN 'Communication Housings'
					ELSE NULL
				END) AS [ASSET_TYPE_COMMUNICATION_HOUSINGS]
			,(CASE
					WHEN FuncLocName LIKE '%50v battery%' THEN '50V DC Power Supplies'
					ELSE NULL
				END) AS [ASSET_TYPE_DC_POWER_SUPPLIES]
			,(CASE
					WHEN FuncLocName LIKE '%battery bank%' THEN 'DC Battery Banks'
					ELSE NULL
				END) AS [ASSET_TYPE_BATTERY_BANKS]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'UTC' OR AZJ_SG_SYSTYPE = 'DTC' THEN 'UTC & DTC'
					ELSE NULL
				END) AS [ASSET_TYPE_UTC_AND_DTC]
			,(CASE
					WHEN [AZJ_SG_EQUIPGRP] = 'CABLES' AND [AZJ_SG_EQUIPTYPE] = 'SINGLEMODE O/FIBRE' OR [AZJ_SG_EQUIPTYPE] = 'SINGLEMODE OPGW CABLE' THEN 'New Fibre Optic Cable' --Should be 'Original Fibre Optic Cable (installed 1985)' based on install date, that isn't here... The old ones have 30-year life, so should be replaced by now if kept to schedule.
					ELSE NULL
				END) AS [ASSET_TYPE_CABLES]
			,(CASE
					WHEN [AZJ_SG_EQUIPTYPE] = 'GUYED STEEL LATTICE TOWER' OR [AZJ_SG_EQUIPTYPE] = 'STEEL LATTICE TOWER' THEN 'Tower (Radio or Microwave)'
					ELSE NULL
				END) AS [ASSET_TYPE_TOWER]
			,(CASE
					WHEN FuncLocID LIKE 'LX%' AND FuncLocID = FuncLocID6 THEN 'Level Crossings (Software)'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_SOFTWARE]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'HOUSINGS' AND ASSET_OWNER = 'AURIZON - NETWORK' AND WorkCenterDesc LIKE '%Control%' AND AZJ_SG_SYSID LIKE '%HOUSING%' THEN 'Communication Housings'
					ELSE NULL
				END) AS [ASSET_TYPE_COMMUNICATION_HOUSING]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.electrical.ringfenced].[vw_FLOCElectrical]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF8 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_UPS] AS VARCHAR(100)) AS [ASSET_TYPE_UPS]
				,CAST([ASSET_TYPE_INTERLOCKINGS] AS VARCHAR(100)) AS [ASSET_TYPE_INTERLOCKINGS]
				,CAST([ASSET_TYPE_TRACK_CIRCUITS] AS VARCHAR(100)) AS [ASSET_TYPE_TRACK_CIRCUITS]
				,CAST([ASSET_TYPE_AXLE_COUNTERS] AS VARCHAR(100)) AS [ASSET_TYPE_AXLE_COUNTERS]
				,CAST([ASSET_TYPE_DRAGGING_EQUIPMENT] AS VARCHAR(100)) AS [ASSET_TYPE_DRAGGING_EQUIPMENT]
				,CAST([ASSET_TYPE_WEIGHBRIDGES] AS VARCHAR(100)) AS [ASSET_TYPE_WEIGHBRIDGES]
				,CAST([ASSET_TYPE_LX_CAMERAS] AS VARCHAR(100)) AS [ASSET_TYPE_LX_CAMERAS]
				,CAST([ASSET_TYPE_LX_PROTECTION] AS VARCHAR(100)) AS [ASSET_TYPE_LX_PROTECTION]
				,CAST([ASSET_TYPE_DATA_LAN] AS VARCHAR(100)) AS [ASSET_TYPE_DATA_LAN]
				,CAST([ASSET_TYPE_RADIO_SYSTEMS] AS VARCHAR(100)) AS [ASSET_TYPE_RADIO_SYSTEMS]
				,CAST([ASSET_TYPE_RADIO_DEDS] AS VARCHAR(100)) AS [ASSET_TYPE_RADIO_DEDS]
				,CAST([ASSET_TYPE_RADIO_POLES] AS VARCHAR(100)) AS [ASSET_TYPE_RADIO_POLES]
				,CAST([ASSET_TYPE_DMR] AS VARCHAR(100)) AS [ASSET_TYPE_DMR]
				,CAST([ASSET_TYPE_TERMINAL_EQUIPMENT] AS VARCHAR(100)) AS [ASSET_TYPE_TERMINAL_EQUIPMENT]
				,CAST([ASSET_TYPE_EQUIPMENT_ROOMS] AS VARCHAR(100)) AS [ASSET_TYPE_EQUIPMENT_ROOMS]
				,CAST([ASSET_TYPE_COMMUNICATION_HOUSINGS] AS VARCHAR(100)) AS [ASSET_TYPE_COMMUNICATION_HOUSINGS]
				,CAST([ASSET_TYPE_DC_POWER_SUPPLIES] AS VARCHAR(100)) AS [ASSET_TYPE_DC_POWER_SUPPLIES]
				,CAST([ASSET_TYPE_BATTERY_BANKS] AS VARCHAR(100)) AS [ASSET_TYPE_BATTERY_BANKS]
				,CAST([ASSET_TYPE_UTC_AND_DTC] AS VARCHAR(100)) AS [ASSET_TYPE_UTC_AND_DTC]
				,CAST([ASSET_TYPE_CABLES] AS VARCHAR(100)) AS [ASSET_TYPE_CABLES]
				,CAST([ASSET_TYPE_TOWER] AS VARCHAR(100)) AS [ASSET_TYPE_TOWER]
				,CAST([ASSET_TYPE_LX_SOFTWARE] AS VARCHAR(100)) AS [ASSET_TYPE_LX_SOFTWARE]
				,CAST([ASSET_TYPE_COMMUNICATION_HOUSING] AS VARCHAR(100)) AS [ASSET_TYPE_COMMUNICATION_HOUSING]
			FROM DF7) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_UPS],
				[ASSET_TYPE_INTERLOCKINGS],
				[ASSET_TYPE_TRACK_CIRCUITS],
				[ASSET_TYPE_AXLE_COUNTERS],
				[ASSET_TYPE_DRAGGING_EQUIPMENT],
				[ASSET_TYPE_WEIGHBRIDGES],
				[ASSET_TYPE_LX_CAMERAS],
				[ASSET_TYPE_LX_PROTECTION],
				[ASSET_TYPE_DATA_LAN],
				[ASSET_TYPE_RADIO_SYSTEMS],
				[ASSET_TYPE_RADIO_DEDS],
				[ASSET_TYPE_RADIO_POLES],
				[ASSET_TYPE_DMR],
				[ASSET_TYPE_TERMINAL_EQUIPMENT],
				[ASSET_TYPE_EQUIPMENT_ROOMS],
				[ASSET_TYPE_COMMUNICATION_HOUSINGS],
				[ASSET_TYPE_DC_POWER_SUPPLIES],
				[ASSET_TYPE_BATTERY_BANKS],
				[ASSET_TYPE_UTC_AND_DTC],
				[ASSET_TYPE_CABLES],
				[ASSET_TYPE_TOWER],
				[ASSET_TYPE_LX_SOFTWARE],
				[ASSET_TYPE_COMMUNICATION_HOUSING])
			) AS unpvt
	),

	DF9 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN FuncLocID LIKE 'LX%' AND FuncLocID = FuncLocID6 THEN 'Level Crossing (Hardware)'
					ELSE NULL
				END) AS [ASSET_TYPE_LX_HARDWARE]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.operationalsystem.ringfenced].[vw_FLOCLevelCrossingExtended]
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF10 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_LX_HARDWARE] AS VARCHAR(100)) AS [ASSET_TYPE_LX_HARDWARE]
			FROM DF9) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_LX_HARDWARE])
			) AS unpvt
	),

	DF11 AS (
		SELECT [FuncLocID]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
			,(CASE
					WHEN AZJ_SG_SYSTYPE = 'INTERLOCKING' AND WorkCenterDesc LIKE '%Control%' THEN 'Processor Based Interlockings'
					ELSE NULL
				END) AS [ASSET_TYPE_INTERLOCKING]
		FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.signalling.ringfenced].[vw_FLOCSignal] 
		WHERE [UserStatusDesc]  = 'In Service'
	),

	DF12 AS(
		SELECT [FuncLocID]
			,[ASSET]
			,[ASSET_TYPE]
			,[FuncLocLinearLengthKM]
			,[LinearRefPattern]
		FROM
		(SELECT [FuncLocID]
				,[FuncLocLinearLengthKM]
				,[LinearRefPattern]
				,CAST([ASSET_TYPE_INTERLOCKING] AS VARCHAR(100)) AS [ASSET_TYPE_INTERLOCKING]
			FROM DF11) D
		UNPIVOT 
			(ASSET FOR ASSET_TYPE IN
				([ASSET_TYPE_INTERLOCKING])
			) AS unpvt
	),

	RESULT AS (
		SELECT * FROM DF2
		UNION ALL
		SELECT * FROM DF4
		UNION ALL
		SELECT * FROM DF6
		UNION ALL
		SELECT * FROM DF8
		UNION ALL
		SELECT * FROM DF10
		UNION ALL
		SELECT * FROM DF12
	)

	SELECT * FROM RESULT;
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_control_system_floc = pd.read_sql_query(sql_query, conn)

all_control_system_floc['ASSET_GROUP'] = 'control_systems'

# # # Display the DataFrame
# all_control_system_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_17612\1078124443.py:384: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_control_system_floc = pd.read_sql_query(sql_query, conn)


In [91]:
# all_traction_power_equipment_floc
sql_query = """
    -- Traction Power Equipment Asset Renewal Python Replication in SQL.
    WITH DF1 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE 
                    WHEN AZJ_SG_EQUIPTYPE = 'HARMONIC FILTER' THEN 'Harmonic Filters'
                    ELSE NULL
                END) AS [ASSET_TYPE_FILTER]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'Transformer' AND AZJ_SG_SYSGRP = 'TRACTION POWER SYSTEM' THEN 'Traction Power Supply Transformers'
                    ELSE NULL
                END) AS [ASSET_TYPE_TRANSFORMER]
            ,(CASE 
                    WHEN AZJ_SG_EQUIPTYPE = 'AUTOTRANSFORMER' THEN 'Autotransformers'
                    ELSE NULL
                END) AS [ASSET_TYPE_AUTOTRANSFORMER]
            ,(CASE 
                    WHEN AZJ_SG_EQUIPTYPE = 'AUXILARY SUPPLY TRANSFORMER' THEN 'Auxiliary Supply Transformers'
                    ELSE NULL
                END) AS [ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'HV Switchgear' AND AZJ_SG_SYSGRP = 'TRACTION POWER SYSTEM' THEN 'Traction Power Systems Switchgear'
                    ELSE NULL
                END) AS [ASSET_TYPE_SWITCHGEAR]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'Isolators' THEN 'Isolator'
                    ELSE NULL
                END) AS [ASSET_TYPE_ISOLATOR]
            ,(CASE 
                    WHEN TechnicalObjectTypeDesc = 'Neutral Section' THEN 'Neutral Sections'
                    WHEN TechnicalObjectTypeDesc = 'Section Insulator' THEN 'Section Insulators'
                    ELSE NULL
                END) AS [ASSET_TYPE_SECTION]
            ,(CASE 
                    WHEN AZJ_SG_SYSGRP = 'TRACTION OVERHEAD SYSTEM' THEN 'Overhead Line System'
                    ELSE NULL
                END) AS [ASSET_TYPE_OVERHEAD]
            ,(CASE 
                    WHEN AZJ_SG_SYSGRP = 'SCADA' AND AZJ_SG_SYSTYPE = 'POWER SUPERVISORY' THEN 'Traction SCADA'
                    ELSE NULL
                END) AS [ASSET_TYPE_SCADA]
        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.electrical.ringfenced].[vw_FLOCElectrical]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF2 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_FILTER] AS VARCHAR(100)) AS [ASSET_TYPE_FILTER]
                ,CAST([ASSET_TYPE_TRANSFORMER] AS VARCHAR(100)) AS [ASSET_TYPE_TRANSFORMER]
                ,CAST([ASSET_TYPE_AUTOTRANSFORMER] AS VARCHAR(100)) AS [ASSET_TYPE_AUTOTRANSFORMER]
                ,CAST([ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER] AS VARCHAR(100)) AS [ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER]
                ,CAST([ASSET_TYPE_SWITCHGEAR] AS VARCHAR(100)) AS [ASSET_TYPE_SWITCHGEAR]
                ,CAST([ASSET_TYPE_ISOLATOR] AS VARCHAR(100)) AS [ASSET_TYPE_ISOLATOR]
                ,CAST([ASSET_TYPE_SECTION] AS VARCHAR(100)) AS [ASSET_TYPE_SECTION]
                ,CAST([ASSET_TYPE_SECTION] AS VARCHAR(100)) AS [ASSET_TYPE_OVERHEAD]
                ,CAST([ASSET_TYPE_SCADA] AS VARCHAR(100)) AS [ASSET_TYPE_SCADA]
            FROM DF1) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_FILTER]
                ,[ASSET_TYPE_TRANSFORMER]
                ,[ASSET_TYPE_AUTOTRANSFORMER]
                ,[ASSET_TYPE_AUXILIARY_SUPPLY_TRANSFORMER]
                ,[ASSET_TYPE_SWITCHGEAR]
                ,[ASSET_TYPE_ISOLATOR]
                ,[ASSET_TYPE_SECTION]
                ,[ASSET_TYPE_OVERHEAD]
                ,[ASSET_TYPE_SCADA])
            ) AS unpvt
    ),

    DF3 AS (
        SELECT [FuncLocID]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
            ,(CASE 
                    WHEN Characteristic = 'ZPM_ISOLATOR_TYPE' AND ([Value] = 'MANUAL' OR [Value] = 'MOTORISED') THEN 'Motorised Isolator Motor Units'
                    ELSE NULL
                END) AS [ASSET_TYPE_MOTORISED_ISOLATORS]

        FROM [myANALYTICS_SP].[bronze.batch.belowrail.asset.ringfenced].[vw_FLOCEQUICharacteristics]
        WHERE [UserStatusDesc]  = 'In Service'
    ),

    DF4 AS(
        SELECT [FuncLocID]
            ,[ASSET]
            ,[ASSET_TYPE]
            ,[FuncLocLinearLengthKM]
            ,[LinearRefPattern]
        FROM
        (SELECT [FuncLocID]
                ,[FuncLocLinearLengthKM]
                ,[LinearRefPattern]
                ,CAST([ASSET_TYPE_MOTORISED_ISOLATORS] AS VARCHAR(100)) AS [ASSET_TYPE_MOTORISED_ISOLATORS]
            FROM DF3) D
        UNPIVOT 
            (ASSET FOR ASSET_TYPE IN
                ([ASSET_TYPE_MOTORISED_ISOLATORS])
            ) AS unpvt
    ),

    RESULT AS (
        SELECT * FROM DF2
        UNION ALL
        SELECT * FROM DF4
    )

    SELECT * FROM RESULT
"""

# Define your server name
server_name = 'myanalytics.aurizon.com.au'

# Establish a connection using Windows Authentication
conn = pyodbc.connect('DRIVER={SQL Server};SERVER=' + server_name + ';Trusted_Connection=yes;')

# Execute the SQL query and load the result into a pandas DataFrame
all_traction_power_equipment_floc = pd.read_sql_query(sql_query, conn)

all_traction_power_equipment_floc['ASSET_GROUP'] = 'traction_power_equipment'

# # # Display the DataFrame
# all_traction_power_equipment_floc

C:\Users\R893859\AppData\Local\Temp\ipykernel_17612\3727245082.py:130: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  all_traction_power_equipment_floc = pd.read_sql_query(sql_query, conn)


In [92]:
all_renewal_floc = pd.concat([all_structure_floc, all_civil_track_floc, all_civil_right_of_way_floc, all_control_system_floc, all_structure_floc, all_traction_power_equipment_floc], ignore_index=True)
all_renewal_floc

,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern,ASSET_GROUP,FuncLocLinearLengthKM
0,CU100204,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN
1,CU100208,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19730101,BW-03ML,civil_structure,NaN
2,CU100218,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19620101,NC-09ML,civil_structure,NaN
3,CU100223,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN
4,CU100231,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19830101,BW-03ML,civil_structure,NaN
...,...,...,...,...,...,...,...,...
56492,ST007780-18,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0
56493,ST006630-18,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0
56494,ST006630-33,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0
56495,ST006630-37,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0


In [93]:
merged_df = pd.merge(all_renewal_floc, all_renewal_wo, how='left', left_on='FuncLocID', right_on='FunctionLocation')
# test_df[test_df['ActualTotalCost'].notna()]

# count = merged_df['ActualTotalCost'].notna().sum()
# print(f"Amount of floc renewal cost we get from WOs: {count}")

merged_df

,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern,ASSET_GROUP,FuncLocLinearLengthKM,FunctionLocation,ActualTotalCost
0,CU100204,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN,CU100204,787.57
1,CU100208,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19730101,BW-03ML,civil_structure,NaN,NaN,NaN
2,CU100218,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19620101,NC-09ML,civil_structure,NaN,NaN,NaN
3,CU100223,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN,NaN,NaN
4,CU100231,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19830101,BW-03ML,civil_structure,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
58323,ST007780-18,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN
58324,ST006630-18,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN
58325,ST006630-33,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN
58326,ST006630-37,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN


In [106]:
# Add new column 'RenewalCost' with multiple conditions (for structures only)
def calculate_renewal_cost(row):
    if pd.notna(row['ActualTotalCost']):
        return row['ActualTotalCost']
    elif row['ASSET_TYPE'] in ['Asset_Type_Bridge', 'Asset_Type_Overbridge']:
        return 1600000
    elif row['ASSET_TYPE'] in ['Asset_Type_Boxculvert', 'Asset_Type_Pipe', 'Asset_Type_Arch', 'Asset_Type_Wall']:
        return 635977
    elif row['ASSET_TYPE'] == 'ASSET_TYPE_SLEEPERS':
        return 697
    elif row['ASSET_TYPE'] == 'ASSET_TYPE_RAIL':
        return 367077
    elif row['ASSET_TYPE'] == 'ASSET_TYPE_CROSSING':
        return 806476
    else:
        return None

merged_df['RenewalCost'] = merged_df.apply(calculate_renewal_cost, axis=1)

merged_df = merged_df.drop_duplicates()
merged_df
# merged_df[merged_df['ASSET'] == '28.0 Tonne Axle Load PSC Sleepers']
# merged_df[merged_df['ASSET_TYPE'] == 'ASSET_TYPE_RAIL']

,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern,ASSET_GROUP,FuncLocLinearLengthKM,FunctionLocation,ActualTotalCost,RenewalCost
0,CU100204,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN,CU100204,787.57,787.57
1,CU100208,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19730101,BW-03ML,civil_structure,NaN,NaN,NaN,635977.00
2,CU100218,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19620101,NC-09ML,civil_structure,NaN,NaN,NaN,635977.00
3,CU100223,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN,NaN,NaN,635977.00
4,CU100231,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19830101,BW-03ML,civil_structure,NaN,NaN,NaN,635977.00
...,...,...,...,...,...,...,...,...,...,...,...
58323,ST007780-18,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN,NaN
58324,ST006630-18,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN,NaN
58325,ST006630-33,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN,NaN
58326,ST006630-37,Motorised Isolator Motor Units,ASSET_TYPE_MOTORISED_ISOLATORS,NaN,NaN,BW-03ML,traction_power_equipment,0.0,NaN,NaN,NaN


In [95]:
# merged_df.to_excel(r'C:\Users\R893859\Aurizon Operations Limited\Data, Reporting & Analytics - DS08 - NSAP2\Data\Floc Renewal Cost\floc_renewal_cost.xlsx', index=False)

In [96]:
test1 = merged_df[merged_df['ASSET_GROUP'] == 'civil_structure']
test1
# test1[test1['RenewalCost'].isna()]

,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern,ASSET_GROUP,FuncLocLinearLengthKM,FunctionLocation,ActualTotalCost,RenewalCost
0,CU100204,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN,CU100204,787.57,787.57
1,CU100208,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19730101,BW-03ML,civil_structure,NaN,NaN,NaN,635977.00
2,CU100218,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19620101,NC-09ML,civil_structure,NaN,NaN,NaN,635977.00
3,CU100223,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19790101,BW-03ML,civil_structure,NaN,NaN,NaN,635977.00
4,CU100231,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19830101,BW-03ML,civil_structure,NaN,NaN,NaN,635977.00
...,...,...,...,...,...,...,...,...,...,...,...
3915,CU400608,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19700101,MA-02FK,civil_structure,NaN,NaN,NaN,635977.00
3916,CU400609,Reinforced Concrete Box Culvert (RCBC) prior t...,Asset_Type_Boxculvert,0,19830101,MA-03FK,civil_structure,NaN,NaN,NaN,635977.00
3917,CU400611,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,None,MA-02FK,civil_structure,NaN,NaN,NaN,635977.00
3918,CU400612,Reinforced Concrete Pipe (RCP),Asset_Type_Pipe,0,19730101,MA-02FK,civil_structure,NaN,NaN,NaN,635977.00


In [97]:
test1[test1.duplicated('FuncLocID', keep=False)].sort_values(by='FuncLocID')

,FuncLocID,ASSET,ASSET_TYPE,STRUCTURE_LENGTH,YEAR_OF_CONSTRUCTION,LinearRefPattern,ASSET_GROUP,FuncLocLinearLengthKM,FunctionLocation,ActualTotalCost,RenewalCost
216,BR101049,Steel Bridge,Asset_Type_Bridge,10.6,19680101,BW-05ML,civil_structure,NaN,NaN,NaN,1600000.0
217,BR101049,Pedestrian Overbridge (Steel),Asset_Type_Overbridge,10.6,19680101,BW-05ML,civil_structure,NaN,NaN,NaN,1600000.0
218,BR101296,Steel Bridge,Asset_Type_Bridge,17,19800101,BW-01ML,civil_structure,NaN,NaN,NaN,1600000.0
219,BR101296,Pedestrian Overbridge (Steel),Asset_Type_Overbridge,17,19800101,BW-01ML,civil_structure,NaN,NaN,NaN,1600000.0
790,BR200326,Steel Bridge,Asset_Type_Bridge,105,19000101,GA-02YR,civil_structure,NaN,NaN,NaN,1600000.0
...,...,...,...,...,...,...,...,...,...,...,...
3040,CU400571,Concrete Pipe (Unreinforced),Asset_Type_Pipe,None,20150101,MA-05CT,civil_structure,NaN,NaN,NaN,635977.0
3039,CU400571,Reinforced Concrete Box Culvert (RCBC) designe...,Asset_Type_Boxculvert,None,20150101,MA-05CT,civil_structure,NaN,NaN,NaN,635977.0
3043,CU400577,Concrete In situ Arch,Asset_Type_Arch,18,20130101,MA-16ML,civil_structure,NaN,NaN,NaN,635977.0
3042,CU400577,Concrete Pipe (Unreinforced),Asset_Type_Pipe,18,20130101,MA-16ML,civil_structure,NaN,NaN,NaN,635977.0
